## Import Libraries

In [2]:
import json
import os
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory

In [3]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

C:\Users\junai\AppData\Local\Temp\ipykernel_8356\2878734244.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)


## Prepare Schema

In [4]:
with open("../schema.json", "r") as f:
    schema = json.load(f)


## Query Generator

In [5]:
from dotenv import load_dotenv
load_dotenv()


True

In [6]:
system_message = SystemMessagePromptTemplate.from_template(
"You are a smart and intelligent assistant for the Pakistan Cricket Chatbot. Your job is to understand natural language questions from users and retrieve accurate cricket statistics from the MySQL database."
"You are also an expert SQL assistant. Only generate safe, syntactically correct SELECT queries based on the provided database schema: {schema}."
"You are strictly prohibited from making any changes to the database. Do NOT generate queries with any of the following keywords: CREATE, ALTER, DROP, TRUNCATE, RENAME, ADD, MODIFY, DELETE, REMOVE, CHANGE."
"If the user asks for such operations, respond politely: 'Sorry, I’m only allowed to fetch data, not modify it.'"
"Only return SQL SELECT statements without any explanations or formatting — just the raw SQL."
).format(schema=schema)


human_message = HumanMessagePromptTemplate.from_template("{input}")

chat  = ChatGroq(
    temperature=0.9,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name = "llama3-70b-8192",
    max_tokens=100
)

def Query_generator(input):
    prompt = ChatPromptTemplate.from_messages([
        system_message,
        human_message
    ])
    llm_chain = LLMChain(llm=chat, prompt=prompt, memory=memory)
    query = llm_chain.run(input)
    cleaned_query = query.replace("```", "").replace("\n", " ").strip()
    return cleaned_query



## DataBase Connection

In [7]:
import os
from dotenv import load_dotenv

# Load .env from parent directory
load_dotenv(dotenv_path="../.env")  # because .env is one level up


True

In [8]:
load_dotenv(dotenv_path="C:/Users/junai/Desktop/chat-bot2/chat-bot/.env")


True

In [ ]:
# pip install mysql-connector-python


In [11]:
import mysql.connector
import os

# Get connection info from environment or directly fill them here
config = {
    'user': os.getenv('DB_USER', 'your_username'),
    'password': os.getenv('DB_PASSWORD', 'your_password'),
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'your_database'),
}


## Execute Query

In [12]:
def execute_query(query):
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return [dict(zip(columns, row)) for row in results]
    except mysql.connector.Error as err:
        return f"MySQL Error: {err}"
    finally:
        cursor.close()
        conn.close()


In [13]:
query = Query_generator("what is the address?")
results = execute_query(query)

print(results)


C:\Users\junai\AppData\Local\Temp\ipykernel_8356\1212160386.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=chat, prompt=prompt, memory=memory)
C:\Users\junai\AppData\Local\Temp\ipykernel_8356\1212160386.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  query = llm_chain.run(input)


MySQL Error: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Sorry, I'm only allowed to fetch data, not provide external information. Please ' at line 1


## Generate Answer

In [ ]:
from langchain.prompts import PromptTemplate


def generate_answer(question, result):
    conversational_prompt = PromptTemplate.from_template(
        "You are a friendly Pakistan Cricket  assistant. Using the ongoing conversation and the result from the database, "
        "respond conversationally and clearly. Be concise and helpful.\n\n"
        "Conversation History:\n{history}\n\n"
        "User Question: {question}\n"
        "Database Result: {result}\n\n"
        "Answer:"
    )
    prompt = conversational_prompt.format(
        history=memory.load_memory_variables({})["history"],
        question=question,
        result=result
    )
    response = chat.invoke(prompt)
    memory.save_context({"input": question}, {"output": response.content})  # save this round
    return response.content


In [16]:
question1 = "Babar azam t20 performance"
query1 = Query_generator(question1)
result1 = execute_query(query1)
print(generate_answer(question1, result1))

I can help you with Babar Azam's T20 performance! According to our database, Babar Azam has played 123 matches, scoring a total of 4145 runs with an average of 41.00. His highest score is 122, and he has 36 fifties and 3 centuries to his name. He has also taken 50 catches in his T20 career.


In [17]:
question2 = "2020 matches?"
query2 = Query_generator(question2)
result2 = execute_query(query2)
print(generate_answer(question2, result2))



I'm happy to help you with the 2020 matches! According to our database, here are the performances of some players in 2020:

* Mohammad Rizwan played 218 matches, scoring 6614 runs with an average of 33.00, and took 139 wickets.
* Nadeem Ghauri played 91 matches, scoring 740 runs with an average of 15.00, and took 120 wickets.
* Sajjad Akbar played


In [18]:
question3 = "Who scored the most runs in ODI matches?"
query3 = Query_generator(question3)
result3 = execute_query(query3)
print(generate_answer(question3, result3))

I can answer that for you! According to our database, the player who scored the most runs in ODI matches is the one with player_id 336, and they have scored a whopping 11,701 runs!


In [19]:
question4 = "List players who have scored more than 5,000 runs in ODIs?"
# No new query — generate from memory
print(generate_answer(question4, ""))

According to our database, the players who have scored more than 5,000 runs in ODIs are:

* Player with ID 336 (11,701 runs)
* Player with ID 253 (6,921 runs)
* Player with ID 217 (6,434 runs)
* Player with ID 183 (5,962 runs)

Let me know if you'd like more information about these players!


In [21]:
import mysql.connector
import os
from dotenv import load_dotenv

# ✅ Load environment variables from .env file
load_dotenv()

# ✅ Connect to the MySQL database
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)

# ✅ Create a cursor object
cursor = conn.cursor()

# ✅ Query to find players with more than 5000 ODI runs
query = """
SELECT p.player_id, p.player_name, o.runs_scored
FROM players p
JOIN odi_match_stats o ON p.player_id = o.player_id
WHERE o.runs_scored > 5000;
"""

# ✅ Execute the query
cursor.execute(query)

# ✅ Fetch the results
results = cursor.fetchall()

# ✅ Display the players
print("Players with more than 5000 ODI runs:\n")
for row in results:
    print(f"Player ID: {row[0]}, Name: {row[1]}, Runs: {row[2]}")

# ✅ Close connection
cursor.close()
conn.close()


Players with more than 5000 ODI runs:

Player ID: 270, Name: Javed Miandad, Runs: 7381
Player ID: 293, Name: Saleem Malik, Runs: 7170
Player ID: 311, Name: Rameez Raja, Runs: 5841
Player ID: 315, Name: Abdur Rauf, Runs: 6564
Player ID: 323, Name: Anwar Ali, Runs: 8823
Player ID: 336, Name: Inzamam0ul0Haq1, Runs: 11701
Player ID: 364, Name: Imad Wasim, Runs: 8064
Player ID: 366, Name: Imran Farhat, Runs: 5080
Player ID: 377, Name: Kashif Raza, Runs: 9720
Player ID: 383, Name: Mansoor Amjad, Runs: 7534
Player ID: 386, Name: Younis Khan, Runs: 7240
Player ID: 397, Name: Mohammad Irfan, Runs: 5122
Player ID: 399, Name: Mohammad Rizwan, Runs: 6614
